In [19]:
import pandas as pd
import requests
import re
import numpy as np
import os
import tensorflow as tf

from bs4 import BeautifulSoup
from math import ceil

import translators.server as ts
from langdetect import detect

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import model_from_json

import warnings
from IPython.display import clear_output
warnings.filterwarnings("ignore")

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36', 'Accept-Language':'pt-BR,pt;q=0.9,en;q=0.8'}
s = requests.Session()

In [3]:
input = 'Xiaomi 11 Lite NE'
text = input + ' amazon'
url = "https://www.google.com/search?q=" + text

request_result=s.get(url).content

soup = BeautifulSoup(request_result,"html.parser")

In [4]:
cont = 0

new_url = ''
reviews = []
produtos = []
for a in soup.find_all('a', href=True):    
    if 'amazon.com' in a['href'] and '/dp/' in a['href']:
        href888 = a['href']
        produtos.append(a.text)
        new_url = href888.replace('/url?q=', '') # remove /url?q=
        url_coments = new_url.split('&')[0].replace('/dp/', '/product-reviews/').replace('.br', '')
        if '//'  in url_coments:
            url_coments = url_coments.split('//')[1]
        reviews.append('https://' + url_coments)

In [6]:
coments = []
cont=0
for r in reviews:
    
    try:
        html = s.get(r + f'/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1', headers=headers).content
        soup = BeautifulSoup(html, 'html.parser')

        a_soupi = soup.find_all('div', class_='a-row a-spacing-base a-size-base')
        
        if len(a_soupi) == 0: # se não encontrar a div, pula para o próximo produto
            continue
        print(f"Analisando produto {produtos[cont]}")
        print(a_soupi[0].text.replace('\n', '').replace('  ', ''))
        i = int(a_soupi[0].text.replace('\n', '').replace('  ', '').replace(',', '').split(' ')[3])/10 # pegando o numero de páginas
        i = ceil(i)

        print(f'Número de páginas: {i}')
        
        if i == 1: # se só tiver uma página roda o for só uma vez
            i+=1

        for j in range(1, i):
            
            html = s.get(r + f'/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber={j}', headers=headers).content
            soup = BeautifulSoup(html, 'html.parser')
            a_soup = soup.find_all('div', class_='a-row a-spacing-small review-data')
            
            for a in a_soup:
                for coment in a.text.split('\n'):
                    if coment != '':
                        coments.append(coment)
    except Exception as e:
        print(e)
        continue

    cont+=1
    print()

Analisando produto Smartphone Mi 11 Lite 5G NE 128gb 8GB RAM - Truffle Black - Pretowww.amazon.com.br › Smartphone-Lite-128gb-8GB-RAM
870 total ratings, 243 with reviews
Número de páginas: 25

Analisando produto Celular Xiaomi Mi 11 lite NE 5G 256Gb Azul Versão Global - Amazonwww.amazon.com.br › Celular-Xiaomi-NE-5G-Versão
1,960 total ratings, 575 with reviews
Número de páginas: 58

Analisando produto Xiaomi 11 Lite 5G NE Smartphone 8+128 Branco | Amazon.com.brwww.amazon.com.br › Xiaomi-Lite-Smartphone-128-Branco
1,960 total ratings, 575 with reviews
Número de páginas: 58

Analisando produto Xiaomi 11 Lite 5G NE Smartphone 8+128 Azul | Amazon.com.brwww.amazon.com.br › Xiaomi-Lite-Smartphone-128-Azul
1,960 total ratings, 575 with reviews
Número de páginas: 58

Analisando produto Xiaomi 11 Lite 5G NE Smartphone 8+128 Preto | Amazon.com.brwww.amazon.com.br › Xiaomi-Lite-Smartphone-128-Preto
1,960 total ratings, 575 with reviews
Número de páginas: 58



In [7]:
coments_translate = []
cont = 1

# pelo fato de demorar muito para traduzir os comentários, se o número de comentários for maior que 100, ele não traduz
if len(coments) > 100:
    coments_translate = coments

else:
    for c in coments:
        print(f"Traduzindo {len(coments)} comentários...")
        print(f"Comentário {cont} de {len(coments)}")
        try:
            coments_translate.append(ts.google(c, to_language='en'))
        except:
            continue
        cont+=1

print(f'{len(coments_translate)} de {len(coments)} comentários')

## Testando modelo

In [9]:
import nltk
import spacy.cli
import en_core_web_sm
from nltk.corpus import stopwords

nltk.download('stopwords')
stopwords_en = stopwords.words("english")

spacy.cli.download("en_core_web_sm")
spc_en = en_core_web_sm.load()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pichau\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [10]:
def limpa_texto(texto):
  '''(str) -> str
  Essa funcao recebe uma string, deixa tudo em minusculo, filtra apenas letras,
  retira stopwords, lemmatiza e retorna a string resultante.
  '''
  texto = texto.lower()

  texto = re.sub(r"[\W\d_]+", " ", texto)

  texto = [pal for pal in texto.split() if pal not in stopwords_en]

  spc_texto = spc_en(" ".join(texto))
  tokens = [word.lemma_ if word.lemma_ != "-PRON-" else word.lower_ for word in spc_texto]
  
  return " ".join(tokens)

In [11]:
try:
    df_alexa = pd.read_csv('df_preprocessed.csv')
    df_alexa.dropna(inplace=True)
    
except:
    print('Dataset preprocessado nao encontrado, criando novo dataset...')
    try:
        df = pd.read_csv('Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')

        df_alexa = df.drop(['id','dateAdded', 'dateUpdated', 'asins', 'keys', 'manufacturer', 'manufacturerNumber', 
        'reviews.date', 'reviews.dateSeen', 'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id',
         'reviews.numHelpful', 'reviews.sourceURLs', 'reviews.username', 'imageURLs', 'primaryCategories', 'categories',
         'brand', 'name', 'sourceURLs'], axis=1)

        df_alexa.fillna('', inplace = True) # para nao ter problemas com nulos na concatenacao

        # concatenando as duas colunas
        df_alexa['verified_reviews'] = df_alexa['reviews.text'] + ' ' + df_alexa['reviews.title']
        # removendo entradas sem texto
        df_alexa = df_alexa[df_alexa['verified_reviews'] != ' ']


        # transformando rating em feedback 0 e 1
        labels = []
        for score in df_alexa['reviews.rating']:
            if score > 3:
                labels.append(1)
            else:
                labels.append(0)

        df_alexa['feedback'] = labels
        # Aplica a funcao nas reviews do dataset
        df_alexa['verified_reviews'] = df_alexa['verified_reviews'].apply(limpa_texto)

        # Salva o dataset preprocessado
        
        df_alexa.to_csv('df_preprocessed.csv', index=False)
    except Exception as e:
        print(e)
        print('Dataset nao encontrado, baixe o dataset e coloque na mesma pasta do codigo')
        exit()
        

## Vectorizer

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
df_alexa['feedback'].value_counts()

1    25543
0     2786
Name: feedback, dtype: int64

In [14]:
texto = df_alexa['verified_reviews']
# Importando o TfidfVectorizer

# Instanciando o TfidfVectorizer
tfidf_vect = TfidfVectorizer()

# Vetorizando
X_tfidf = tfidf_vect.fit_transform(texto)

## Modelo

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf.toarray(), df_alexa['feedback'], test_size = 0.2)

In [16]:
try:
    # load json and create model
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    classifier = model_from_json(loaded_model_json)
    # load weights into new model
    classifier.load_weights("model.h5")
    print("Loaded model from disk")

    # evaluate loaded model on test data
    classifier.compile(optimizer='Adam', loss='binary_crossentropy', metrics = ['accuracy'])
except:
    # create model and save
    classifier = tf.keras.models.Sequential()
    classifier.add(tf.keras.layers.Dense(units = 10, activation='relu', input_shape=(X_train.shape[1],)))
    classifier.add(tf.keras.layers.Dense(units = 10, activation='relu'))
    classifier.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

    classifier.compile(optimizer='Adam', loss='binary_crossentropy', metrics = ['accuracy'])

    # fit model
    epochs_hist = classifier.fit(X_train, y_train, epochs=100, batch_size=50,  verbose=2, validation_split=0.2)


    # serialize model to JSON
    model_json = classifier.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    classifier.save_weights("model.h5")
    print("Saved model to disk")

Loaded model from disk


In [17]:
# classification_report
y_pred_test = (classifier.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test,y_pred_test))

178/178 [==============================] - 0s 992us/step
              precision    recall  f1-score   support

           0       0.89      0.93      0.91       558
           1       0.99      0.99      0.99      5108

    accuracy                           0.98      5666
   macro avg       0.94      0.96      0.95      5666
weighted avg       0.98      0.98      0.98      5666



In [18]:

if len(coments_translate) == 0:
    print('Nenhum comentário para ser classificado')
else:
    c = tfidf_vect.transform(coments_translate).toarray()
    predict = (classifier.predict(c) > 0.5).astype(int)

    cont_pos = 0
    cont_neg = 0
    for p in predict:
        if p == 1:
            cont_pos += 1
        else:
            cont_neg += 1

    print(f'Positivos: {cont_pos} | Negativos: {cont_neg}')

    indexes_neg = np.where(predict == 0)[0] # obtendo indexes dos comentarios negativos

    if len(indexes_neg) == 0:
        print('Nenhum comentario negativo encontrado')
    else:
        print("\n Comentarios negativos: \n")
        for i in indexes_neg:
            print(ts.google(coments_translate[i], to_language='pt'), '\n')

82/82 [==============================] - 0s 985us/step
Positivos: 2247 | Negativos: 347

 Comentarios negativos: 

A mídia não pôde ser carregada. 

Eu amo o telefone!Os alto -falantes estavam com uma imagem alta, mas não funcionaria com as redes telefônicas tristemente. 

Pelo preço, este é um ótimo telefone, não poderia estar mais feliz! 

Este telefone não funcionará nos EUA na ATT 

O telefone é excelente para o preço, mas só funciona com a AT&T e T Mobile, aqui é a minha dica se você tiver uma dessas operadoras, não vá à loja ou chama o serviço de fantasia para ativação, eles recusarão apenas o cartão SIM eInsira -o no telefone e configure as configurações de APN e é lembrado que este telefone e qualquer telefone chinês não receberão serviço 5G nos EUA 

Eu chego super rápido excelente qualidade 

Câmera com imagem em alta resolução! 

Celular Leve com qualidade de imagem excelente, bem nitida. Desempenho otimo, não trava, mesmo com varios apps abertos. O principal na minha opnião